# Imports

In [ ]:
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from google.cloud import bigquery
import os
from datetime import date, datetime
import pandas as pd


In [ ]:
"""
    parameters received from papermill

    these are the parameters that are passed to the script
    above is the default values
    they can be overridden by the user when running the script
"""

BASE_URL = "https://www.gov.br/anp/pt-br"
B100_SALES = f"{BASE_URL}/assuntos/distribuicao-e-revenda/comercializacao-de-biodiesel"
RAW_PATH = "data/raw/b100_sales"

bucket_name = os.getenv("GOOGLE_BUCKET_NAME")

In [ ]:
"""
Get file download URL
"""

response = requests.get(B100_SALES, verify=False)
soup = BeautifulSoup(response.content, "html.parser")
if soup is None:
    raise Exception("Failed to retrieve data from the URL.")
title_text = 'Volumes Comercializados de Biodiesel'
title_h3 = soup.find('h3', string=title_text)
jump_p = title_h3.find_next_sibling()
jump_p = jump_p.find_next_sibling()
year = jump_p.find_next_sibling()

data_by_year = {}

current_year = year.get_text(strip=True)
data_by_year[current_year] = []

next_elem = year.find_next_sibling()
while next_elem:
    if next_elem.name == 'ul':
        for li in next_elem.find_all('li'):
            a_tag = li.find('a')
            if a_tag and 'href' in a_tag.attrs:
                link = a_tag['href']
                text = a_tag.get_text(strip=True)
                li_text = li.get_text(strip=True)


                start = li_text.find('Atualizado em ')
                if start != -1:
                    li_text = li_text[start + len('Atualizado em '):-1].strip()

                data_by_year[current_year].append({'text': text, 'link': link, 'updated_date': li_text})
            else:
                print("Elemento <li> nÃ£o contÃ©m um link.")
        next_elem = next_elem.find_next_sibling()
    else:
        if next_elem.name == 'h3' and next_elem.get_text(strip=True).isdigit():
            current_year = next_elem.get_text(strip=True)
            if current_year not in data_by_year:
                data_by_year[current_year] = []
            next_elem = next_elem.find_next_sibling()
            continue
        else:
            break


In [ ]:
current_years = {
	"2023",
	"2024",
	"2025"
}
all_dataframes = []
for year in current_years:
    if year in data_by_year and data_by_year[year]:
        print(f"Processing year: {year}")
        
        download_url = data_by_year[year][0]['link']
        response = requests.get(download_url, verify=False)
        
        if response.status_code != 200:
            print(f"Failed to download file for year {year}: {response.status_code}")
            continue
            
        file_bytes = response.content
        file_buffer = BytesIO(file_bytes)
        
        B100_BRONZE_COLUMNS_MAPPING = {
            "2023": {
                "Mês/Ano": "vb100_dat_compra",
                "Raiz\nCNPJ": "vb100_num_cnpj",
                "Razão Social": "vb100_txt_razao_social",
                "Quantidade\nde Produto\n(m³)": "vb100_qtd_volume"
            },
            "2024": {
                "Data": "vb100_dat_compra",
                "Raiz de CNPJ do Distribuidor": "vb100_num_cnpj",
                "Razão Social do Distribuidor": "vb100_txt_razao_social",
                "Razão Social do Produtor": "vb100_nom_produtor",
                "CNPJ do Produtor": "vb100_num_produtor_cnpj",
                "Volume (m3)": "vb100_qtd_volume"
            },
            "2025": {
                "Data": "vb100_dat_compra",
                "Raiz de CNPJ do Distribuidor": "vb100_num_cnpj",
                "Razão Social do Distribuidor": "vb100_txt_razao_social",
                "Razão Social do Produtor": "vb100_nom_produtor",
                "CNPJ do Produtor": "vb100_num_produtor_cnpj",
                "Volume (m3)": "vb100_qtd_volume"
            }
        }
        
        map_to_use = B100_BRONZE_COLUMNS_MAPPING.get(year, B100_BRONZE_COLUMNS_MAPPING["2024"])
        
        try:
            df_year = pd.read_excel(file_buffer, header=2, usecols=lambda x: 'Unnamed' not in x)
            df_year = df_year.rename(columns=map_to_use)
            
            df_year['vb100_dat_compra'] = pd.to_datetime(df_year['vb100_dat_compra'], format='%Y%m')
            df_year["vb100_num_cnpj"] = df_year["vb100_num_cnpj"].astype(str).str.zfill(8)
            df_year['vb100_dat_compra'] = df_year['vb100_dat_compra'].dt.date
            df_year['vb100_qtd_volume'] = df_year['vb100_qtd_volume'].astype(float)
            df_year['vb100_dat_criacao'] = pd.Timestamp.now(tz='America/Sao_Paulo')
            
            if int(year) < 2024:
                df_year["vb100_nom_produtor"] = None
                df_year["vb100_num_produtor_cnpj"] = None
            else:
                df_year["vb100_num_produtor_cnpj"] = df_year["vb100_num_produtor_cnpj"].astype(str).str.zfill(14)
            
            all_dataframes.append(df_year)
            print(f"Successfully processed {len(df_year)} rows for year {year}")
            
        except Exception as e:
            print(f"Error processing data for year {year}: {str(e)}")
    else:
        print(f"No data found for year {year}")

if all_dataframes:
    df = pd.concat(all_dataframes, ignore_index=True)
    print(f"Combined dataset has {len(df)} total rows")
else:
    raise Exception("No data was processed for any year")

In [ ]:
"""
    change columns data types
"""

df['vb100_dat_compra'] = pd.to_datetime(df['vb100_dat_compra'], format='%Y%m')
df["vb100_num_cnpj"] = df["vb100_num_cnpj"].astype(str).str.zfill(8)
df['vb100_dat_compra'] = df['vb100_dat_compra'].dt.date
df['vb100_qtd_volume'] = df['vb100_qtd_volume'].astype(float)
df['vb100_dat_criacao'] = pd.Timestamp.now(tz='America/Sao_Paulo')

if int(current_year) < 2024:
    df["vb100_nom_produtor"] = None
    df["vb100_num_produtor_cnpj"] = None
else:
    df["vb100_num_produtor_cnpj"] = df["vb100_num_produtor_cnpj"].astype(str).str.zfill(14)

In [ ]:
"""
    insert data into BigQuery with date-based partitioning
"""

client = bigquery.Client()
project_id = os.getenv("GOOGLE_CLOUD_PROJECT")
bq_dataset = "rw_ext_anp"
table_name = "venda_b100"

table_id = f"{project_id}.{bq_dataset}.{table_name}"

job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)

partition_key = date.today().strftime('%Y%m%d')

partitioned_table_id = f"{table_id}${partition_key}"
print(f"Inserting data for partition: {partition_key}")
print(f"Total rows to insert: {len(df)}")

try:
    job = client.load_table_from_dataframe(
        df, partitioned_table_id, job_config=job_config
    )
    job.result()
    print(f"Data for {partition_key} inserted successfully.")
    
    df['year'] = pd.to_datetime(df['vb100_dat_compra']).dt.year
    summary = df.groupby('year').size()
    print("Data summary by year:")
    for year, count in summary.items():
        print(f"  {year}: {count} rows")
        
except Exception as e:
    print(f"Error inserting data for {partition_key}: {str(e)}")

print("Data insertion completed!")